# Paso 1
## Configuración de Spark 

Vamos a trabajar con Python y Spark utilizando PySpark, que es la API de Spark para Python. 

Esto nos permitirá manejar grandes volúmenes de datos de manera eficiente y realizar análisis complejos. 

Vamos a configurar nuestro entorno y ejecutar algunos comandos básicos para entender cómo funciona.


In [1]:
# Instalamos pyspark
!pip install pyspark

# Instalamos findspark
!pip install findspark

In [2]:
# Iniciando el contexto de Spark
import pyspark
import findspark
findspark.init()

In [3]:
# Usamos PySpark para inicializar el contexto spark.  
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

### Contexto Spark y Sesión Spark

En este ejercicio, vamos a:
- Crear el Spark Context
- Iniciar la sesión Spark 

#### Primero empezamos con la creación del Spark Context

In [4]:
# Esto no es necesario ejecutarlo
# pip install --upgrade pyspark

In [5]:
# Crear una configuración de Spark
conf = SparkConf().setAppName("Ejemplo Básico de DataFrames con Spark").setMaster("local")

# Crear un Spark Context
sc = SparkContext(conf=conf)

# Crear una Spark Session
spark = SparkSession \
    .builder \
    .appName("Ejemplo Básico de DataFrames con Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/06/11 22:33:11 WARN Utils: Your hostname, eurecat-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/11 22:33:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 22:33:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### En este caso tenemos dos posibles problemas
#### Problema: JAVA_HOME is not set

Si tienen este problema: **JAVA_HOME is not set** es necesario abrir la terminal e instalar java
Para esto podemos ejecutar en la terminal:

```
java --version

sudo apt install openjdk-8-jre-headless

pass: eurecat

ingresar: s
```

Ahora si volvemos a **Visual Studio Code**



#### Problema: ValueError: Cannot run multiple SparkContexts at once
Problema: **ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at C:\Users\lucia.alvarez\AppData\Local\Temp\ipykernel_13740\2225446927.py:5**

Debemos reutilizar el SparkContext existente

Comprobar si hay un SparkContext existente y reutilizarlo

```
if 'sc' in locals():
    sc.stop()
```

Crear una configuración de Spark

```
conf = SparkConf().setAppName("Ejemplo Básico de DataFrames con Spark").setMaster("local")
```

Crear un Spark Context

```
sc = SparkContext(conf=conf)
```

Crear una Spark Session
```
spark = SparkSession \
    .builder \
    .appName("Ejemplo Básico de DataFrames con Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
```

#### Ahora vamos a verificar la Spark Session

In [6]:
# Verificar si la Spark Session está activa
if 'spark' in locals() and isinstance(spark, SparkSession):
    print("SparkSession está activa y lista para usar.")
else:
    print("SparkSession no está activa. Por favor, crea una SparkSession.")

SparkSession está activa y lista para usar.


## Creación de RDD y Transformaciones
En este ejercicio trabajamos con Resilient Distributed Datasets (RDDs). Los RDDs son la abstracción primitiva de datos de Spark y utilizamos conceptos de la programación funcional para crear y manipular RDDs.

#### 01. Crear un RDD
Creamos un RDD que tiene enteros del 10 al 40.

In [7]:
data = range(10, 41)

# Imprime el primer elemento del iterador
print("Primer elemento:", data[0])
print("Longitud del conjunto de datos:", len(data))


Primer elemento: 10
Longitud del conjunto de datos: 31


In [8]:
# Crear un RDD a partir de los datos
dataRDD = sc.parallelize(data, 4)


#### 02. Transformaciones en el RDD

Una transformación es una operación sobre un RDD que da como resultado un nuevo RDD. 

El RDD contendrá una serie de transformaciones, o instrucciones de cálculo, que sólo se llevarán a cabo cuando se llame a una acción. En esta transformación, reducimos cada elemento del RDD en 5. 

Además, filtramos el RDD para que sólo contenga elementos menores a 25.


In [9]:
# Reducimos cada elemento del RDD en 5
subRDD = dataRDD.map(lambda x: x - 5)

In [10]:
# Filtramos el RDD para que solo contenga elementos menores a 25
filteredRDD = subRDD.filter(lambda x: x < 25)

#### 03: Acciones en nuestro RDD
Aplicamos acciones a nuestro RDD transformado para obtener y contar los resultados.

In [11]:
# Acciones en el RDD
print("Elementos del RDD filtrado:", filteredRDD.collect())

Elementos del RDD filtrado: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [12]:
print("Cantidad de elementos en el RDD filtrado:", filteredRDD.count())

Cantidad de elementos en el RDD filtrado: 20


#### 04: Almacenamiento de Datos en Caché
Almacenar datos en caché en Spark mejora el rendimiento de las operaciones repetitivas en un RDD.

In [13]:
import time 

# Crear un nuevo RDD con un rango de datos
testRDD = sc.parallelize(range(1000, 51000), 4)


# Almacenar el RDD en caché
testRDD.cache()

# Primera evaluación: esto desencadenará la evaluación y el almacenamiento en caché
t1 = time.time()
count1 = testRDD.count()
dt1 = time.time() - t1
print("Tiempo para el primer conteo (sin caché):", dt1)


# Segunda evaluación: ahora los datos están cacheados
t2 = time.time()
count2 = testRDD.count()
dt2 = time.time() - t2
print("Tiempo para el segundo conteo (con caché):", dt2)


Tiempo para el primer conteo (sin caché): 1.0601565837860107
Tiempo para el segundo conteo (con caché): 0.4849233627319336


#### Finalización de la Sesión de Spark


In [14]:
# Detener la sesión de spark
spark.stop() 

Fin de este práctico